Credits # Source

Thank you https://github.com/RakaMaru/Faceswap_Google_Colab_Master/blob/master/Faceswap_Google_Colab_Master.ipynb for modified notebook and
https://github.com/deepfakes/faceswap for source code

# More Credits

Thank you Faceswap Community

*   https://faceswap.dev/
*   https://forum.faceswap.dev/
*   The other Github Google Colab notebooks which inspired this one



# Check Your System

In [ ]:
#@title Use this to check the assigned GPU

!cat /etc/os-release

def install_dependencies():
  !ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi;
  !pip install gputil;
  !pip install psutil;
  !pip install humanize;%%capture


def printm():
 GPUs = GPU.getGPUs()

 if len(GPUs) == 0:
  print("No GPU available.")
  return

 gpu = GPUs[0]
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

from IPython.utils import io
from google.colab import drive
import psutil
import humanize
import os
 
with io.capture_output() as captured:
  install_dependencies()
print("Dependencies installed.")


import GPUtil as GPU

printm()

# Setup Faceswap

In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
#@title Grab the latest Faceswap
!rm -rf faceswap
!git clone --single-branch --branch r1.0 https://github.com/deepfakes/faceswap.git

In [ ]:
#@title Install Tensorflow

!pip install -r faceswap/requirements_nvidia.txt


# Extract Faces

In [ ]:
#COMP365
# Extracting input video's face
!python3 faceswap/faceswap.py extract -i "/content/drive/My Drive/AI/cashme.mp4" -o "/content/drive/My Drive/AI/cashme" -D s3fd -A fan -nm none -min 0 -l 0.4 -een 1 -sz 256 -si 0 -L INFO

In [ ]:
#COMP365
# Extracting Prof Skalak's face
!python3 faceswap/faceswap.py extract -i "/content/drive/My Drive/AI/skalak2.mp4" -o "/content/drive/My Drive/AI/skalak2" -D s3fd -A fan -nm none -min 0 -l 0.4 -een 1 -sz 256 -si 0 -L INFO

# Run Training

Training code from https://colab.research.google.com/github/seranus/faceswap-notebooks/blob/master/faceswap_trainer.ipynb#scrollTo=_UFW54uqbtgv

In [ ]:
#Threading stuff stolen from: 
# https://stackoverflow.com/questions/32081926/a-new-thread-for-running-a-cell-in-ipython-jupyter-notebook

from time import sleep,time
from IPython.display import display,HTML
import base64
from threading import Thread

# new faceswap version
image_path = "faceswap/training_preview.jpg"

# old faceswap
#image_path = "faceswap/_sample_training.jpg"
#image_path = "faceswap/_sample_training using <OriginalHighRes: v=2.7, enc=ORIGINAL, encoder_dim=1024, img_shape=128x128>, bs=16.jpg"
!touch '{image_path}'

class PreviewImg:
  def __init__(self):
    self.cancel = False
    self.fig = display(HTML('<img src="https://dummyimage.com/1024x618/000/ffffff&text=Preview+Loading..." />'), display_id=True)
    print("Created display.")
  
  def update(self):
    print("Updating display.")
    with open(image_path, 'rb') as image:
      encoded = str(base64.b64encode(image.read()))[2:-1]
      self.fig.update(HTML('<img src="data:image/jpg;base64,' + encoded + '" width="1024" height="618" />'))
      # print("Updated display.")
    
  def task(self):
    while not self.cancel:
      sleep(5 * 60) #Every 5 minutes.
      if not self.fig:
        print("Figure doesn't exist.")
        break
      self.update()

In [ ]:
!echo "Preview image size is reduced 3 times right click and save the image to view it in full size"

prev_img = PreviewImg()
thread = Thread(target=prev_img.task)
thread.start()

!python3 faceswap/faceswap.py train -A '/content/drive/My Drive/AI/cashme' -ala '/content/drive/My Drive/AI/cashme.fsa' -B '/content/drive/My Drive/AI/skalak2' -alb '/content/drive/My Drive/AI/skalak2.fsa' -m '/content/drive/My Drive/AI/model10000' -t 'villain' -bs 16 -s 330 -w -nl -it 10000

# Convert

In [ ]:
#COMP365
!python3 faceswap/faceswap.py convert -i '/content/drive/MyDrive/AI/video-1606520221.mp4' -o '/content/drive/My Drive/AI/output10000.mp4' -al '/content/drive/MyDrive/AI/cashme.fsa' -m '/content/drive/MyDrive/AI/model10000' -c "avg-color" -M "none" -w "ffmpeg" -osc "100" -l "0.4" -j "0" -L "INFO"

# Analysis 



In [ ]:
#@title Grab the latest Faceswap
!rm -rf DeepFake-Experiment

!git clone https://github.com/raphminkyu/DeepFake-Experiment.git

In [ ]:
!pip install -r DeepFake-Experiment/requirements.txt

In [ ]:
python3 DeepFake-Experiment/main.py